In [ ]:
!pip install gen3_validator

In [ ]:
resolver.schema

In [ ]:
resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.schema

In [ ]:
resolver.nodes

In [ ]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.props import PropExtractor

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

In [ ]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.props import PropExtractor

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

node_props = PropExtractor(resolver.schema_resolved['unaligned_reads_file.yaml'])


print(node_props.schema_name)

# return prop names for the schema
prop_names = node_props.get_prop_names()
print(prop_names)

# return the data types for the properties
types = {}
for prop_name in prop_names:
    types[prop_name] = node_props.get_data_type(prop_name)
print(types)

# return the description for the properties
descriptions = {}
for prop_name in prop_names:
    descriptions[prop_name] = node_props.get_description(prop_name)
print(descriptions)

In [2]:
node_props = PropExtractor(resolver.schema_resolved["sample.yaml"])
props_list = node_props.extract_properties()
# Nest the properties under the node name, so node name is not repeated in each property
node_name = node_props.schema_name
props_list_json = {
    node_name: [
        {
            "prop_name": p.prop_name,
            "data_type": p.data_type,
            "description": p.description
        }
        for p in props_list
    ]
}
import json
print(json.dumps(props_list_json, indent=2))

{
  "sample": [
    {
      "prop_name": "type",
      "data_type": "string",
      "description": null
    },
    {
      "prop_name": "id",
      "data_type": "string | pattern = ^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$",
      "description": "A 128-bit identifier. Depending on the mechanism used to generate it, it is either guaranteed to be different from all other UUIDs/GUIDs generated until 3400 AD or extremely likely to be different. Its relatively small size lends itself well to sorting, ordering, and hashing of all sorts, storing in databases, simple allocation, and ease of programming in general.\n"
    },
    {
      "prop_name": "submitter_id",
      "data_type": "string",
      "description": "A project-specific identifier for a node. This property is the calling card/nickname/alias for a unit of submission. It can be used in place of the UUID for identifying or recalling a node.\n"
    },
    {
      "prop_name": "state",
      "data_ty

In [ ]:
import pandas as pd
node_props = PropExtractor(resolver.schema_resolved["sample.yaml"])
props_list = node_props.extract_properties()
props_list_pd = pd.DataFrame([p.__dict__ for p in props_list])
props_list_pd

In [ ]:
import pandas as pd

    node_props = PropExtractor(resolver.schema_resolved[f"{node}"])
    props_list = node_props.extract_properties()
    node_templates[node] = pd.DataFrame([p.__dict__ for p in props_list])

node_templates

In [ ]:
node_templates["lipidomics_file"]

# testing out templates module

In [ ]:
from gen3_metadata_templates.templates import *

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()
make_node_template_json(resolver.schema_resolved, "lipidomics_file.yaml")\
    
make_node_template_xlsx(resolver.schema_resolved, "lipidomics_file.yaml")

INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00?\x00a]I:O\x01\x00\x00\x8f\x04\x00\x00\x13\x00\x00\x00[Content_Types].xml\xad\x94\xcbn\xc20\x10E\xf7\xfd\x8a\xc8\xdb*1tQU\x15\x81E\x1f\xcb\x16\xa9\xf4\x03\\{B,\x1c\xdb\xf2\x0c\x14\xfe\xbe\x93\xf0P[Q\xa0\x82M\xacd\xee\xdcs\xc7\x8e<\x18-\x1b\x97- \xa1\r\xbe\x14\xfd\xa2\'2\xf0:\x18\xeb\xa7\xa5x\x9f<\xe7w"CR\xde(\x17<\x94b\x05(F\xc3\xab\xc1d\x15\x013n\xf6X\x8a\x9a(\xdeK\x89\xba\x86Fa\x11"x\xaeT!5\x8a\xf85MeTz\xa6\xa6 oz\xbd[\xa9\x83\'\xf0\x94S\xeb!\x86\x83G\xa8\xd4\xdcQ\xf6\xb4\xe4\xcf\xeb \t\x1c\x8a\xeca-lY\xa5P1:\xab\x15q].\xbc\xf9E\xc97\x84\x82;;\r\xd66\xe25\x0b\x84\xdcKh+\x7f\x036}\xaf\xbc3\xc9\x1a\xc8\xc6*\xd1\x8bjX%M\xd0\xe3\x14"J\xd6\x17\x87]\xf6\xc4\x0cUe5\xb0\xc7\xbc\xe1\x96\x02\xda@\x06L\x1e\xd9\x12\x12Y\xd8e>\xc8\xd6!\xc1\xff\xe1\xdb=j\xbbO$.\x9dDZ9\xc0\xb3G\xc5\x98@\x19\xac\x01\xa8q\xc5\xda\xf4\x08\x99\xf8\x7f\x82\xf5\xb3\x7f6\xbf\xb39\x02\xfc\x0ci\xf6\x11\xc2\xec\xd2\xc3\xb6k\xd1(\xebO\xe0wb\x94\xddr\xfe\xd4?\x83\xec\xfc\x8f\x1dy\xad

In [3]:
resolver.get_all_node_pairs()

INFO:gen3_validator.dict:Retrieving all node pairs, excluding specified nodes.
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: demographic.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: demographic.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: project.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: project.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: serum_marker_assay.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: serum_marker_assay.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: alignment_workflow.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: alignment_workflow.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: imaging_file.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: imaging_file.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: lipidomics_assay.yaml
INFO

[('timepoint', 'demographic'),
 ('program', 'project'),
 ('sample', 'serum_marker_assay'),
 ('unaligned_reads_file', 'alignment_workflow'),
 ('timepoint', 'imaging_file'),
 ('core_metadata_collection', 'imaging_file'),
 ('sample', 'lipidomics_assay'),
 ('metabolomics_assay', 'metabolomics_file'),
 ('core_metadata_collection', 'metabolomics_file'),
 ('project', 'acknowledgement'),
 ('timepoint', 'medical_history'),
 ('timepoint', 'blood_pressure_test'),
 ('sample', 'genomics_assay'),
 ('aligned_reads_file', 'variant_file'),
 ('variant_workflow', 'variant_file'),
 ('core_metadata_collection', 'variant_file'),
 ('subject', 'timepoint'),
 ('serum_marker_assay', 'serum_marker_file'),
 ('core_metadata_collection', 'serum_marker_file'),
 ('sample', 'proteomics_assay'),
 ('timepoint', 'sample'),
 ('genomics_assay', 'unaligned_reads_file'),
 ('core_metadata_collection', 'unaligned_reads_file'),
 ('aligned_reads_file', 'aligned_reads_index_file'),
 ('alignment_workflow', 'aligned_reads_index_fil

In [4]:
from gen3_validator.dict import *
get_min_node_path(
    edges=resolver.get_all_node_pairs(),
    target_node="lipidomics_file"
    )

INFO:gen3_validator.dict:Retrieving all node pairs, excluding specified nodes.
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: demographic.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: demographic.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: project.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: project.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: serum_marker_assay.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: serum_marker_assay.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: alignment_workflow.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: alignment_workflow.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: imaging_file.yaml
INFO:gen3_validator.dict:Retrieving links and ID for node: imaging_file.yaml
INFO:gen3_validator.dict:Finding upstream and downstream nodes for: lipidomics_assay.yaml
INFO

Graph root node(s): ['program']


PathInfo(path=['program', 'project', 'subject', 'timepoint', 'sample', 'lipidomics_assay', 'lipidomics_file'], steps=6)

In [ ]:
from gen3_metadata_templates.templates import *
from gen3_validator.dict import *

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()


node_order = get_min_node_path(
    edges=resolver.get_all_node_pairs(),
    target_node="lipidomics_file"
    ).path

xlsx_dict = {}



for node in node_order:
    xlsx_dict[node] = make_node_template_xlsx(resolver.schema_resolved, node)

combine_xlsx_sheets(xlsx_dict, "lipidomics_file.xlsx")

INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

Graph root node(s): ['program']
program


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [2]:
xlsx_dict

{'demographic.yaml': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00?\x00a]I:O\x01\x00\x00\x8f\x04\x00\x00\x13\x00\x00\x00[Content_Types].xml\xad\x94\xcbn\xc20\x10E\xf7\xfd\x8a\xc8\xdb*1tQU\x15\x81E\x1f\xcb\x16\xa9\xf4\x03\\{B,\x1c\xdb\xf2\x0c\x14\xfe\xbe\x93\xf0P[Q\xa0\x82M\xacd\xee\xdcs\xc7\x8e<\x18-\x1b\x97- \xa1\r\xbe\x14\xfd\xa2\'2\xf0:\x18\xeb\xa7\xa5x\x9f<\xe7w"CR\xde(\x17<\x94b\x05(F\xc3\xab\xc1d\x15\x013n\xf6X\x8a\x9a(\xdeK\x89\xba\x86Fa\x11"x\xaeT!5\x8a\xf85MeTz\xa6\xa6 oz\xbd[\xa9\x83\'\xf0\x94S\xeb!\x86\x83G\xa8\xd4\xdcQ\xf6\xb4\xe4\xcf\xeb \t\x1c\x8a\xeca-lY\xa5P1:\xab\x15q].\xbc\xf9E\xc97\x84\x82;;\r\xd66\xe25\x0b\x84\xdcKh+\x7f\x036}\xaf\xbc3\xc9\x1a\xc8\xc6*\xd1\x8bjX%M\xd0\xe3\x14"J\xd6\x17\x87]\xf6\xc4\x0cUe5\xb0\xc7\xbc\xe1\x96\x02\xda@\x06L\x1e\xd9\x12\x12Y\xd8e>\xc8\xd6!\xc1\xff\xe1\xdb=j\xbbO$.\x9dDZ9\xc0\xb3G\xc5\x98@\x19\xac\x01\xa8q\xc5\xda\xf4\x08\x99\xf8\x7f\x82\xf5\xb3\x7f6\xbf\xb39\x02\xfc\x0ci\xf6\x11\xc2\xec\xd2\xc3\xb6k\xd1(\xebO\xe0wb\x94\xddr\xfe\xd4?\x83

In [2]:
node_order

['program',
 'project',
 'subject',
 'timepoint',
 'sample',
 'lipidomics_assay',
 'lipidomics_file']

In [1]:
from gen3_metadata_templates.templates import *

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

generate_xlsx_templates(resolver, "lipidomics_file", "lipidomics_file.xlsx")

INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

Graph root node(s): ['program']
program
project
subject
timepoint
sample
lipidomics_assay
lipidomics_file


In [3]:
[node.replace('.yaml', '') for node in resolver.nodes]

['demographic',
 'project',
 'serum_marker_assay',
 'alignment_workflow',
 'imaging_file',
 'lipidomics_assay',
 'metabolomics_file',
 'acknowledgement',
 'medical_history',
 '_definitions',
 '_settings',
 'blood_pressure_test',
 'genomics_assay',
 'variant_file',
 'timepoint',
 'program',
 'serum_marker_file',
 'proteomics_assay',
 'sample',
 'unaligned_reads_file',
 '_terms',
 'aligned_reads_index_file',
 'variant_workflow',
 'proteomics_file',
 'exposure',
 'metabolomics_assay',
 'lipidomics_mapping_file',
 'lipidomics_file',
 'aligned_reads_file',
 'lab_result',
 'medication',
 'publication',
 'subject',
 'core_metadata_collection']

In [ ]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.templates import *
resolver = ResolveSchema("../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

In [1]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.templates import *
resolver = ResolveSchema("../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()
df = make_node_template_pd(resolver.schema_resolved["lipidomics_file.yaml"])
format_df = format_node_xlsx(resolver, df)
format_df

INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

lipidomics_file-submitter_id,lipidomics_assays-submitter_id,state,project_id,created_datetime,updated_datetime,file_name,file_size,file_format,md5sum,cv,data_category,data_format,data_type,imputed_data,individual_or_sum,lipidomic_unit
0,string,None,string,None,None,string,integer,string,string | pattern = ^[a-f0-9]{32}$,number,enum,enum,enum,enum,enum,enum
1,Foreign key unique identifier,The current state of the object.\n,Unique ID for any specific defined piece of wo...,A combination of date and time of day in the f...,A combination of date and time of day in the f...,The name (or part of a name) of a file (of any...,The size of the data file (object) in bytes.\n,The format of the data file object.\n,The 128-bit hash value expressed as a 32 digit...,Coefficient of variation (%),Broad categorization of the contents of the da...,The format of the data in this data file,The type of data in this data file,Does the lipidomic file contain imputed data f...,Does the lipidomic file contain data for indiv...,Units of measurement captured in the lipidomic...


In [2]:
format_df.columns = format_df.iloc[0]
format_df = format_df.drop(format_df.index[0]).reset_index(drop=True)
format_df

type,id,lipidomics_file-submitter_id,state,project_id,created_datetime,updated_datetime,file_name,file_size,file_format,md5sum,...,ga4gh_drs_uri,lipidomics_assays,core_metadata_collections,cv,data_category,data_format,data_type,imputed_data,individual_or_sum,lipidomic_unit
0,string | pattern = ^[a-fA-F0-9]{8}-[a-fA-F0-9]...,string,None,string,None,None,string,integer,string,string | pattern = ^[a-f0-9]{32}$,...,string,None,None,number,enum,enum,enum,enum,enum,enum
1,A 128-bit identifier. Depending on the mechani...,A project-specific identifier for a node. This...,The current state of the object.\n,Unique ID for any specific defined piece of wo...,A combination of date and time of day in the f...,A combination of date and time of day in the f...,The name (or part of a name) of a file (of any...,The size of the data file (object) in bytes.\n,The format of the data file object.\n,The 128-bit hash value expressed as a 32 digit...,...,DRS URI as defined by GA4GH DRS spec for point...,None,None,Coefficient of variation (%),Broad categorization of the contents of the da...,The format of the data in this data file,The type of data in this data file,Does the lipidomic file contain imputed data f...,Does the lipidomic file contain data for indiv...,Units of measurement captured in the lipidomic...


In [ ]:
pk_name = "lipidomics_file-submitter_id"
fk_names_list = ["lipidomics_assays"]

exclude_cols = ["type", "id", "object_id", "file_state", "error_type", "ga4gh_drs_uri", 'core_metadata_collections']

colnames = format_df.columns.tolist()

# Refactored: create a set for fast lookup, then build ordered_colnames
priority_cols = [pk_name] + fk_names_list
priority_set = set(priority_cols)
ordered_colnames = priority_cols + [col for col in colnames if col not in priority_set]
ordered_colnames

format_df = format_df[ordered_colnames]
format_df

type,lipidomics_file-submitter_id,lipidomics_assays,id,state,project_id,created_datetime,updated_datetime,file_name,file_size,file_format,...,error_type,ga4gh_drs_uri,core_metadata_collections,cv,data_category,data_format,data_type,imputed_data,individual_or_sum,lipidomic_unit
0,string,None,string | pattern = ^[a-fA-F0-9]{8}-[a-fA-F0-9]...,None,string,None,None,string,integer,string,...,enum,string,None,number,enum,enum,enum,enum,enum,enum
1,A project-specific identifier for a node. This...,None,A 128-bit identifier. Depending on the mechani...,The current state of the object.\n,Unique ID for any specific defined piece of wo...,A combination of date and time of day in the f...,A combination of date and time of day in the f...,The name (or part of a name) of a file (of any...,The size of the data file (object) in bytes.\n,The format of the data file object.\n,...,Type of error for the data file object.\n,DRS URI as defined by GA4GH DRS spec for point...,None,Coefficient of variation (%),Broad categorization of the contents of the da...,The format of the data in this data file,The type of data in this data file,Does the lipidomic file contain imputed data f...,Does the lipidomic file contain data for indiv...,Units of measurement captured in the lipidomic...


In [ ]:
from gen3_metadata_templates.props import *

node_props = PropExtractor(resolver.schema_resolved['demographics.yaml'])
node_props.get_data_type

# Trying again


In [3]:
df

,node_name,prop_name,data_type,description
0,lipidomics_file,type,string,None
1,lipidomics_file,id,string | pattern = ^[a-fA-F0-9]{8}-[a-fA-F0-9]...,A 128-bit identifier. Depending on the mechani...
2,lipidomics_file,submitter_id,string,A project-specific identifier for a node. This...
3,lipidomics_file,state,None,The current state of the object.\n
4,lipidomics_file,project_id,string,Unique ID for any specific defined piece of wo...
5,lipidomics_file,created_datetime,None,A combination of date and time of day in the f...
6,lipidomics_file,updated_datetime,None,A combination of date and time of day in the f...
7,lipidomics_file,file_name,string,The name (or part of a name) of a file (of any...
8,lipidomics_file,file_size,integer,The size of the data file (object) in bytes.\n
9,lipidomics_file,file_format,string,The format of the data file object.\n


In [1]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.templates import *
resolver = ResolveSchema("../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()
df = make_node_template_pd(resolver.schema_resolved["lipidomics_file.yaml"])


INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

In [51]:
df
df['prop_name'].to_list()

['type',
 'id',
 'submitter_id',
 'state',
 'project_id',
 'created_datetime',
 'updated_datetime',
 'file_name',
 'file_size',
 'file_format',
 'md5sum',
 'object_id',
 'file_state',
 'error_type',
 'ga4gh_drs_uri',
 'lipidomics_assays',
 'core_metadata_collections',
 'cv',
 'data_category',
 'data_format',
 'data_type',
 'imputed_data',
 'individual_or_sum',
 'lipidomic_unit']

In [2]:
resolver.get_node_link("project.yaml")

INFO:gen3_validator.dict:Retrieving links and ID for node: project.yaml


('project',
 [{'name': 'programs',
   'backref': 'projects',
   'label': 'member_of',
   'target_type': 'program',
   'multiplicity': 'many_to_one',
   'required': True}])

In [5]:
from gen3_metadata_templates.props import *
from gen3_metadata_templates.templates import *
from gen3_validator.resolve_schema import ResolveSchema

resolver = ResolveSchema("../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()


exclude_columns = [
    "type",
    "id",
    "state",
    "object_id",
    "file_state",
    "error_type",
    "ga4gh_drs_uri",
    "core_metadata_collections",
    "created_datetime",
    "updated_datetime"
]
 

node_order = get_min_node_path(
    edges=resolver.get_all_node_pairs(),
    target_node="lipidomics_file"
    ).path

xlsx_dict = {}
for node in node_order:
    df = make_node_template_pd(resolver=resolver, node=node, exclude_columns=exclude_columns)
    if df is not None:
        xlsx_dict[node] = pd_to_xlsx_mem(df, node)


combine_xlsx_sheets(xlsx_dict, "lipidomics_file.xlsx")


INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

Graph root node(s): ['program']
subject
timepoint


INFO:gen3_metadata_templates.templates:Combined xlsx file saved to lipidomics_file.xlsx


sample
lipidomics_assay
lipidomics_file


# This is the final usage

In [ ]:
from gen3_metadata_templates.templates import *
from gen3_validator.resolve_schema import ResolveSchema

resolver = ResolveSchema("../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

generate_xlsx_template(resolver=resolver, target_node="unaligned_reads_file", output_filename="../examples/output_templates/unaligned_reads_file.xlsx")

INFO:gen3_validator.dict:Initializing DataDictionary with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.dict:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.dict:Successfully read JSON schema.
INFO:gen3_validator.dict:Retrieving node names from schema.
INFO:gen3_validator.dict:Splitting schema into individual node schemas.
INFO:gen3_validator.dict:Split schema into individual node schemas.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _definitions.yaml
INFO:gen3_validator.resolve_schema:Retrieved definitions schema.
INFO:gen3_validator.dict:Retrieving schema for schema ID: _terms.yaml
INFO:gen3_validator.resolve_schema:Retrieved terms schema.
INFO:gen3_validator.resolve_schema:Resolving references in schema.
INFO:gen

Graph root node(s): ['program']
subject
timepoint
sample
genomics_assay
unaligned_reads_file
